In [ ]:
import pandas as pd
from tqdm import tqdm
import numpy as np
from rank_bm25 import BM25Okapi

/Users/rishikgupta/Documents/A&M Courses/ISR 670/Project/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
df = pd.read_csv('data/job_postings_cleaned.csv')  # Load your dataset
df.head()

,company_name,title,description,salary,location,formatted_experience_level,skills_desc,work_type,zip_code,job_matching
0,Essentia Health,Long Term Care Administrator,Looking to Make a Difference in Someone’s Life...,110000.0,"Virginia, MN",Director,NaN,FULL_TIME,55792.0,Long Term Care Administrator Looking to Make a...
1,STIIIZY,Retail Procurement Assistant,We are seeking a proactive and detail-oriented...,58240.0,"Los Angeles, CA",Associate,NaN,FULL_TIME,90001.0,Retail Procurement Assistant We are seeking a ...
2,Net2Source Inc.,Quality Control Inspector,Title: Product Quality Inspector Location: Alp...,104000.0,"Alpharetta, GA",Associate,NaN,CONTRACT,30004.0,Quality Control Inspector Title: Product Quali...
3,Swoon,Creative Project Manager - 79439,Our client is a Fortune 100 company & leading ...,70720.0,"Chicago, IL",Associate,NaN,CONTRACT,60601.0,Creative Project Manager - 79439 Our client is...
4,Murphy USA,CASHIER (full-time & part-time opportunities),Job Posting\n\nAs one of the largest national ...,28080.0,"Lapeer, MI",Entry level,NaN,PART_TIME,48446.0,CASHIER (full-time & part-time opportunities) ...


In [9]:
tokenized_jobs = [job_text.split() for job_text in df['job_matching']]
bm25 = BM25Okapi(tokenized_jobs)

In [12]:
resume_df = pd.read_csv('data/resume_queries.csv')

resume_texts = resume_df['Resume_str'].tolist()

# for each resume text, get the top 10 job matches
results = {}
for resume_text in tqdm(resume_texts):
    resume_tokens = resume_text.lower().split()  # basic tokenization
    scores = bm25.get_scores(resume_tokens)
    
    top_indices = np.argsort(scores)[::-1][:10]
    top_jobs = df.iloc[top_indices].copy()
    top_jobs['bm25_score'] = [scores[i] for i in top_indices]

    results[resume_text] = top_jobs

100%|██████████| 20/20 [00:23<00:00,  1.17s/it]


In [15]:
flattened_rows = []

for resume_text, top_matches_df in results.items():
    # Add the resume text as a new column, repeated for each row
    temp_df = top_matches_df.copy()
    temp_df.insert(0, 'resume_text', resume_text)
    flattened_rows.append(temp_df)

# Concatenate all into one DataFrame
final_df = pd.concat(flattened_rows, ignore_index=True)

# Save to CSV
final_df.to_csv('BM25.csv', index=False)